<a href="https://colab.research.google.com/github/vishnusankar2203/ore-classification/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow keras numpy matplotlib opencv-python


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define dataset path (adjust if needed)
DATASET_PATH = "/content/drive/MyDrive/Minerals Identification Dataset"

# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation to prevent overfitting
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load training & validation data
train_generator = datagen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, subset="training"
)
val_generator = datagen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, subset="validation"
)

# Number of classes (ore types)
NUM_CLASSES = len(train_generator.class_indices)
print(f"Number of ore classes: {NUM_CLASSES}")


Found 768 images belonging to 7 classes.
Found 189 images belonging to 7 classes.
Number of ore classes: 7


In [ ]:
#Train & Save AlexNet Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define AlexNet architecture
alexnet_model = Sequential([
    Conv2D(96, (11, 11), strides=4, activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D((3, 3), strides=2),
    Conv2D(256, (5, 5), activation="relu", padding="same"),
    MaxPooling2D((3, 3), strides=2),
    Conv2D(384, (3, 3), activation="relu", padding="same"),
    Conv2D(384, (3, 3), activation="relu", padding="same"),
    Conv2D(256, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((3, 3), strides=2),
    Flatten(),
    Dense(4096, activation="relu"),
    Dropout(0.5),
    Dense(4096, activation="relu"),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation="softmax")
])

# Compile model
alexnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
alexnet_model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save model to Google Drive
alexnet_model.save("/content/drive/MyDrive/models/alexnet_model.h5")
print("✅ AlexNet model saved successfully!")



Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 161s 7s/step - accuracy: 0.1656 - loss: 2.3245 - val_accuracy: 0.2487 - val_loss: 1.8009
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 155s 6s/step - accuracy: 0.2552 - loss: 1.6617 - val_accuracy: 0.3598 - val_loss: 1.4759
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 154s 6s/step - accuracy: 0.3449 - loss: 1.5500 - val_accuracy: 0.3915 - val_loss: 1.2947
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 162s 7s/step - accuracy: 0.4076 - loss: 1.4442 - val_accuracy: 0.3704 - val_loss: 1.4488
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 164s 7s/step - accuracy: 0.3863 - loss: 1.3988 - val_accuracy: 0.3704 - val_loss: 1.3727
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 163s 7s/step - accuracy: 0.4106 - loss: 1.3491 - val_accuracy: 0.4497 - val_loss: 1.2823
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 163s 7s/step - accuracy: 0.4446 - loss: 1.2837 - val_accuracy: 0.4074 - val_loss: 1.3521
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 163s 7s/step - accuracy: 0.4128 - loss: 1.2960 - val_accuracy: 0.3968 - v

✅ AlexNet model saved successfully!


In [ ]:
#Train & Save VGG16 Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load VGG16 (pre-trained)
vgg16_base = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
vgg16_base.trainable = False  # Freeze base layers

# Build VGG16 model
vgg16_model = Sequential([
    vgg16_base,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    Dense(NUM_CLASSES, activation="softmax")
])

# Compile & Train
vgg16_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
vgg16_model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save VGG16 model
vgg16_model.save("/content/drive/MyDrive/models/vgg16_model.h5")
print("✅ VGG16 model saved successfully!")



Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 627s 26s/step - accuracy: 0.2790 - loss: 1.8428 - val_accuracy: 0.4444 - val_loss: 1.4685
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 613s 26s/step - accuracy: 0.5263 - loss: 1.4314 - val_accuracy: 0.5820 - val_loss: 1.2409
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 612s 26s/step - accuracy: 0.5913 - loss: 1.2131 - val_accuracy: 0.6561 - val_loss: 1.0789
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 633s 27s/step - accuracy: 0.6423 - loss: 1.0462 - val_accuracy: 0.6825 - val_loss: 0.9750
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 606s 25s/step - accuracy: 0.7074 - loss: 0.9162 - val_accuracy: 0.7037 - val_loss: 0.9213
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 607s 25s/step - accuracy: 0.7299 - loss: 0.8702 - val_accuracy: 0.6984 - val_loss: 0.8743
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 607s 25s/step - accuracy: 0.7264 - loss: 0.8387 - val_accuracy: 0.7831 - val_loss: 0.8262
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 625s 26s/step - accuracy: 0.7729 - loss: 0.7490 - val_accuracy: 0.

✅ VGG16 model saved successfully!


In [ ]:
#Train & Save Xception Mode7l
from tensorflow.keras.applications import Xception

# Load Xception model
xception_base = Xception(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
xception_base.trainable = False  # Freeze base layers

# Build Xception model
xception_model = Sequential([
    xception_base,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    Dense(NUM_CLASSES, activation="softmax")
])

# Compile & Train
xception_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
xception_model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save Xception model
xception_model.save("/content/drive/MyDrive/models/xception_model.h5")
print("✅ Xception model saved successfully!")



Epoch 1/10


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/"))